In [1]:
# adds parent dir to python path
import sys
sys.path.insert(0, '..')

import os

import numpy as np
import sep
import matplotlib.pyplot as plt
%matplotlib auto

from astropy.io import fits

import gphelper as gp
import ImageTools as it
import DataTools as dt

Using matplotlib backend: Qt5Agg


In [2]:
def make_nonnegative(img):
    epsilon = 0 if len(img[img<0])==0 else img[img>0].min()
    return img + abs(img.min()) + epsilon

def get_random_spheroids(num=10):
    with open('../spheroids', 'r') as f:
        spheroids = np.array(f.readlines())
    
    if num==-1:
        num = len(spheroids)
    
    selected = np.random.choice(spheroids, num, replace=False)
    
    data_dir = os.path.join(os.getenv('HOME'), 'Documents/astro_data/orig_images')
    fmask = 'GDS_{}_{}.fits'
    f_string = os.path.join(data_dir, fmask)

    sources = []
    for s in selected:
        img = fits.getdata(f_string.format(s.strip(), 'h'))
        segmap = fits.getdata(f_string.format(s.strip(), 'segmap'))
        img_id = int(s.split('_')[1])
        sources.append((img.copy(), segmap.copy(), img_id))
        del img 
        del segmap
    
    return sources

In [62]:
def convergence_update(re, is_converged, int_at_re, int_at_limit):
    tolerance = 1e-3
    
    if (int_at_re/int_at_limit-0.5 > tolerance):
        re = re**0.995
    elif (int_at_re/int_at_limit-0.5 < -tolerance):
        re = re**1.01
    else:
        is_converged = True
        
    return re, is_converged

def check_stop_condition(count, max_attempts, re, rs, fs):
    stop = False
    
    if count==max_attempts:
        print('Max Attempts Reached')
        stop = True
        re_idx = ((np.cumsum(fs)/fs.sum())-0.5).argmin()
        re = rs[re_idx]
        
    elif 5*re >= 42:
        print('Overgrown Image')
        stop = True
        re_idx = np.square((np.cumsum(fs)/fs.sum())-0.5).argmin()
        re = rs[re_idx]
        
    return re, stop

def segmap_too_small(src_map):
    area = src_map.sum()
    r = np.sqrt(area/np.pi)
    return r<=15

def closest_r(rs, r):
    r_idx = np.square(rs-r).argmin()
    return rs[r_idx]

spheroids = get_random_spheroids(num=5)
num_converged, total, num_too_small  = 0, len(spheroids), 0
found_res, rs_re, fs_ie = [], [], []

for img, segmap, img_id in spheroids:
    src_map = segmap==img_id
    array_sep_likes = img.byteswap().newbyteorder()
    bkg = sep.Background(array_sep_likes, mask=src_map,bw=10,bh=10)
    img_sub = make_nonnegative(img-bkg)
    

    cx, cy = it.img_center(img, src_map)
    xs, ys = np.meshgrid(np.arange(img.shape[0]), np.arange(img.shape[1]).T)
    rs = np.sqrt(np.square(cx-xs) + np.square(cy-ys))
    
    if segmap_too_small(src_map):
        print(f'{img_id} segmap too small')
        #take measurement within segmap only
        rs = rs[src_map].flatten()
        fs = img_sub[src_map].flatten()
        
        sorted_rs = np.argsort(rs)
        rs = rs[sorted_rs]
        fs = fs[sorted_rs]
        fs[fs<0] = 0
        
        fs_int = np.cumsum(fs) 
        
        re_idx = np.argmin(np.square(fs_int/fs.sum()-0.5))
        re = rs[re_idx]
        ie = fs[re_idx]
        
        fs /= ie
        rs /= re
        
        fs_int = np.cumsum(fs) / fs.sum()
        
    else:
        # find using iterative method
        rs = rs.flatten()
        fs = img_sub.flatten()

        sorted_rs = np.argsort(rs)
        rs = rs[sorted_rs]
        fs = fs[sorted_rs]

        fs[fs<0] = 0

        fs_int = np.cumsum(fs) 

        re = closest_r(rs, 0.05 * rs.max())
        is_converged = False
        max_attempts = 1000
        count = 0

        while not is_converged:
            int_limit = 5 * re
            int_limit = closest_r(rs, 5*re)
            int_at_re = fs[rs==re]
            int_at_limit = fs[rs==int_limit]
            
            re, is_converged = convergence_update(re, 
                                                  is_converged,
                                                  int_at_re,
                                                  int_at_limit)
            re = closest_r(rs, re)
            
            if not is_converged:
                re, is_converged = check_stop_condition(count, 
                                                        max_attempts, 
                                                        re,
                                                        rs,
                                                        fs)

            count += 1
            
        ie = fs[rs==re].mean()
        rs /= re
        
        fs /= ie
        fs_int = np.cumsum(fs) / fs.sum()
        #rs = rs[rs<=5]
        
    rs_re.append(rs)
    fs_ie.append(fs_int)
        
    found_res.append(re)
        
    if len(spheroids)<5:
        plt.figure()
        plt.title(img_id)
        plt.plot(rs, fs_int)
        plt.vlines([1], fs_int.min(), 1)
        plt.hlines([0.5], 0, rs.max())
                   
        #plt.vlines([re], fs_sub_int.min(), fs_sub_int.max(), color='g', label='$R_e$')
        #plt.vlines([int_limit], fs_sub_int.min(), fs_sub_int.max(), color='r', label="$5R_e$")
        #plt.legend()
        plt.show()

12827 segmap too small
Max Attempts Reached
Max Attempts Reached
11203 segmap too small
14992 segmap too small


In [60]:
hist_res = found_res
plt.hist(hist_res, bins=50)
plt.show()


In [63]:
for r, f in zip(rs_re, fs_ie):
    plt.plot(r, f, '.')

plt.show()